<p><strong><font size="6">WalOUS project</font></strong></p>

<p><strong><font size="6">A_Compute_LcProp_and_RnppDensity_By_CaPa</font></strong></p>

WALOUS_LU - Copyright (C) <2020> <Université catholique de Louvain (UCLouvain), Belgique
					 Université Libre de Bruxelles (ULB), Belgique
					 Institut Scientifique de Service Public (ISSeP), Belgique
					 Service Public de Wallonie (SWP), Belgique >
						 							
	
List of the contributors to the development of WALOUS_LU: see LICENSE file.
Description and complete License: see LICENSE file.
	
This program (WALOUS_LU) is free software:
you can redistribute it and/or modify it under the terms of the
GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option)
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program (see COPYING file).  If not,
see <http://www.gnu.org/licenses/>.

--------
Jupyter Notebook containing the preprocessing steps consisting of: 
- Computing the proportion and mode of land cover (COSW product) classes for each cadastral parcel (zonal stats).
- Creating the raster layer with classes of neighbourhood density and computing mode of neighbourhood density classes for each cadastral parcel.

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Define working environment

**Import libraries**

In [ ]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import glob

**Add folder with SCR provided belong to this notebook**

In [ ]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [ ]:
run ../SRC/config.py

In [ ]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [ ]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

**GRASS GIS Python libraries**

In [ ]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Other functions**

In [ ]:
# Import function for GRASS GIS mapset checking and launching
from grass_database import check_gisdb, check_location, check_mapset, working_mapset, launch_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for importation of r.zonal.classes csv output into a GRASS GIS table
from grass_processing import rzonalclasses_sql_insert

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [ ]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])

# Compute proportion of LC classes in cadastral blocs

## Import land cover (raster)

In [ ]:
# Create mapset 
start_import = start_processing()
launch_mapset("FUSIONS")

# Create a list of paths to files with a specific name prefix
list_file = glob.glob(os.path.join(data['landcover_folder'][1],"ocsol2018final_simplified_31370*.tif"))
print("There are %s .tif files in the folder"%len(list_file))

# Create a list of tuple with GRASS GIS layer name for raster and their path in the computer drive 
list_of_lc_raster = []
[list_of_lc_raster.append((os.path.splitext(a)[0].split(os.sep)[-1],a)) for a in list_file]

# Import individual rasters (for each tile)
print("Importation of %s files..."%len(list_file))
for rast in list_of_lc_raster:
    gscript.run_command('r.in.gdal', overwrite=True, input=rast[1] , output=rast[0])

# If mutliple files, create virtual raster, else (single file) rename it.
if len(list_file) > 1:
    print("Creation of VRT")
    gscript.run_command('r.buildvrt', overwrite=True, 
                        input=",".join([a[0] for a in list_of_lc_raster]), 
                        output=data['landcover_folder'][0])
else: 
    print("Rename input raster")
    gscript.run_command('g.rename', overwrite=True, 
                            raster="%s,%s"%(list_of_lc_raster[0][0],data['landcover_folder'][0]))
    
# Apply color
gscript.run_command('r.colors', map=data['landcover_folder'][0], rules=data['color_file'])
print_processing_time(start_import, "Import achieved in ")

## Import cadastral blocs (vector)

In [ ]:
# Create and launch mapset 
launch_mapset("CAPA")

In [ ]:
# Import vector
start_import = start_processing()
gscript.run_command('v.import', overwrite=True, input=data['capa'][1], output=data['capa'][0])
print_processing_time(start_import, "Import achieved in ")

In [ ]:
# Rasterize cadastral blocs
start = start_processing()
gscript.run_command('g.mapsets', mapset='FUSIONS', operation='add')
gscript.run_command('g.region', vector=data['capa'][0], align=data['landcover_folder'][0])
gscript.run_command('v.to.rast', overwrite=True, input=data['capa'][0], output=data['capa'][0],
                    use='cat', memory=10000)
print_processing_time(start, "Rasterisation achieved in ")

## Compute LC proportion by CaPa

In [ ]:
# Check if add-on is already installed in the computer and install it not yet installed
check_install_addon("r.zonal.classes")

In [ ]:
### Compute LC class proportions in cadastral plots
# Create mapset 
start = start_processing()
launch_mapset("CAPA")
gscript.run_command('g.mapsets', quiet=True, mapset='FUSIONS', operation='add')
gscript.run_command('g.region', raster=data['capa'][0], align=data['landcover_folder'][0])
# Compute LC proportions
tmp_csv = "%s_rzonalclasses"%gscript.tempfile() # Path to temporary file output
ouput_csv = data['lc_capa'][1]
gscript.run_command('r.zonal.classes', overwrite=True, zone_map=data['capa'][0], 
                    raster=data['landcover_folder'][0], prefix='lc', csvfile=tmp_csv)
print_processing_time(start, "Proportions of LC computed in achieved in ")

In [ ]:
### Import the csv output of r.zonal.classes in GRASS SQlite database
start_import = start_processing()
# Load csv content in python dictionnary
incsv = open(tmp_csv, 'r')
reader = csv.reader(incsv, delimiter='|')
header = next(reader)
value_dict = {row[0]:row[1:] for row in reader}
incsv.close()
# Insert SQL
table_name = "lc_prop"
rzonalclasses_sql_insert(table_name, header, value_dict, overwrite=True)

### Table 'capa_with_prop'
table_name = "capa_with_prop"
sql_query = gscript.tempfile()
fsql = open(sql_query, 'w')
fsql.write('BEGIN TRANSACTION;\n')
if gscript.db_table_exist(table_name):
        fsql.write('DROP TABLE %s;\n'%table_name)
create_statement = 'CREATE TABLE %s AS '%table_name
create_statement += 'SELECT a.CAPAKEY, b.* FROM capa AS a '
create_statement += 'JOIN lc_prop AS b ON a.cat=b.cat;\n'
fsql.write(create_statement)
fsql.write('END TRANSACTION;')
fsql.close()
gscript.run_command('db.execute', input=sql_query, quiet=True)
# Export to csv
if not os.path.exists(os.path.split(data['lc_capa'][1])[0]):
    os.makedirs(os.path.split(data['lc_capa'][1])[0])    
gscript.run_command('db.select', overwrite=True, sql="SELECT * FROM %s"%table_name,
                    output=data['lc_capa'][1])
print_processing_time(start_import, "Computation of LC classes proportions achieved in ")

# Compute population density on a 200m surrounding buffer (10 meters spatial resolution)

## Import RNPP population points (vector)

In [ ]:
# Create mapset 
launch_mapset("RNPP")

In [ ]:
# Import vector
start_import = start_processing()
gscript.run_command('v.import', overwrite=True, input=data['rnpp'][1], output=data['rnpp'][0])
print_processing_time(start_import, "Import achieved in ")

## Compute raster of sum of population on a 200m surrounding neighborhood

In [ ]:
# Define parameters
raster_resolution = 10
buffer_diameter = 400
attrib_column = 'MS_POPULAT'

In [ ]:
# Create mapset 
launch_mapset("POP_DENSITY")

In [ ]:
# Give access to other mapsets
gscript.run_command('g.mapsets', quiet=True, mapset='RNPP', operation='add')
# Define computational region
gscript.run_command('g.region', flags='ap', vector=data['rnpp'][0], res=raster_resolution)

In [ ]:
# Execute v.neighbors.stats
start_import = start_processing()
gscript.run_command('v.neighbors.stats', overwrite=True, input=data['rnpp'][0], output='rnpp_popsum', method='sum', 
                    size=buffer_diameter, points_column=attrib_column)
print_processing_time(start_import, "Processing achieved in ")

## Reclass into 4 population density classes

In [ ]:
# Reclassify into density classes
start_import = start_processing()
formula = "density_classes=if(isnull(rnpp_popsum),0,if(rnpp_popsum<80,1,if(rnpp_popsum<250,2,if(rnpp_popsum<500,3,4))))"
gscript.mapcalc(formula, overwrite=True)
print_processing_time(start_import, "Processing achieved in ")

## Export tiff to folder

In [ ]:
file_name = "%s_res_%sm_neighbor_%sm.tif"%(data['rnpp'][0],raster_resolution,int(buffer_diameter/2))
export_path = os.path.join(os.path.split(data['rnpp_neighbor'][1])[0],file_name)
print("Tiff file will be saved on the following location: %s"%export_path)

In [ ]:
# Execute v.neighbors.stats
start_import = start_processing()
gscript.run_command('r.out.gdal', overwrite=True, input='density_classes',
                    output=export_path, format='GTiff', createopt='COMPRESS=DEFLATE', overviews='2')
print_processing_time(start_import, "Processing achieved in ")

## Compute main density class by CaPa

In [ ]:
## Compute main density class by cadastral plot
start = start_processing()
launch_mapset("CAPA")
gscript.run_command('g.mapsets', quiet=True, mapset='POP_DENSITY', operation='add')
# Compute class mode
tmp_csv = "%s_rzonalclasses"%gscript.tempfile() # Path to temporary file output
ouput_csv = data['rnpp_neighbor'][0]
gscript.run_command('r.zonal.classes', overwrite=True, zone_map=data['capa'][0], 
                    raster='density_classes', prefix='rnpp_200m',
                    statistics='mode', csvfile=tmp_csv)
print_processing_time(start, "Modal value of density classes computed in ")

In [ ]:
### Import the csv output of r.zonal.classes in GRASS SQlite database
start_import = start_processing()
# Load csv content in python dictionnary
incsv = open(tmp_csv, 'r')
reader = csv.reader(incsv, delimiter='|')
header = next(reader)
value_dict = {row[0]:row[1:] for row in reader}
incsv.close()
# Insert SQL
table_name_1 = "lc_prop"
SqlInsert(table_name_1, header, value_dict, overwrite=True)

### Table 'capa_with_prop'
table_name_2 = "capa_with_prop"
sql_query = gscript.tempfile()
fsql = open(sql_query, 'w')
fsql.write('BEGIN TRANSACTION;\n')
if gscript.db_table_exist(table_name_2):
        fsql.write('DROP TABLE %s;\n'%table_name_2)
create_statement = 'CREATE TABLE %s AS '%table_name_2
create_statement += 'SELECT a.CAPAKEY, b.* FROM capa AS a '
create_statement += 'JOIN %s AS b ON a.cat=b.cat;\n'%table_name_1
fsql.write(create_statement)
fsql.write('END TRANSACTION;')
fsql.close()
gscript.run_command('db.execute', input=sql_query, quiet=True)
# Export to csv
if not os.path.exists(os.path.split(data['rnpp_neighbor'][1])[0]):
    os.makedirs(os.path.split(data['rnpp_neighbor'][1])[0])    
gscript.run_command('db.select', overwrite=True, sql="SELECT * FROM %s"%table_name_2,
                    output=data['rnpp_neighbor'][1])
print_processing_time(start_import, "Csv file exported in ")